In [ ]:
# import dependencies
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import time
dense_size=64

# load the data set
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train,x_test=x_train/255,x_test/255

# normalise the data
for i in range(x_train.shape[0]):
    x_train[i]-=np.mean(x_train[i],axis=(0,1))
    x_train[i]/=np.std(x_train[i],axis=(0,1))
for i in range(x_test.shape[0]):
    x_test[i]-=np.mean(x_test[i],axis=(0,1))
    x_test[i]/=np.std(x_test[i],axis=(0,1))
# set the seed
tf.random.set_seed(1)


# choose Parameter
learning_rate = 1e-2
batch_size = 64
epochs = 800

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(batch_size)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = layers.Flatten()
    self.d1 = layers.Dense(dense_size, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(5e-5))
    self.d2 = layers.Dense(dense_size, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(5e-5))
    self.d3 = layers.Dense(dense_size, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(5e-5))
    self.d4 = layers.Dense(10, activation='softmax',kernel_regularizer=tf.keras.regularizers.L2(5e-5))

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    x = self.d3(x)
    return self.d4(x)
model = MyModel()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
#train function
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss=loss_fn(labels, predictions)
    loss+=sum(model.losses)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
#test function
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_fn(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [ ]:
history = np.zeros((epochs, 4))
time_stemp = time.monotonic()
time_stemp_0 = time_stemp

for epoch in range(epochs):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  history[epoch] = np.array([train_loss.result(), train_accuracy.result(), test_loss.result(), test_accuracy.result()])
  time_temp = time.monotonic()
  time_diff = time_temp - time_stemp
  time_stemp = time_temp
  time_total = time_temp - time_stemp_0
  TREM = (epochs - epoch+1)*time_total/(epoch+1)
  print(
    f'Epoch {epoch + 1}, '
    f'{time_diff:.0f}s, '
    f'Loss: {train_loss.result():.2e}, '
    f'Accuracy: {train_accuracy.result():.2e}, '
    f'Test Loss: {test_loss.result():.2e}, '
    f'Test Accuracy: {test_accuracy.result():.2e}, '
    f'TREM: {TREM:.0f}s'
    )

In [ ]:
model_str='fcc_model_64r'

# saves the model for later use
model.save('saved_model/'+model_str)
np.save('history/history_'+model_str,history)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
plt.rc('font', size=18)
plt.rcParams['figure.figsize'] = [10, 7]

In [ ]:
plt.plot(1 - history[:,1], label = "training error")
plt.plot(history[:,0], label = "training loss")


plt.title("Entering the Exploration Phase with reg")
#plt.ylabel(r"$S_{v,i}(t)$")
plt.xlabel(r"$t$")
plt.legend()
plt.yscale("log")